<a href="https://colab.research.google.com/github/thaopham03/evaluating_fan_effects_in_large_language_models/blob/main/fan_effects_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import warnings

import pandas as pd
import numpy as np
import ast

from scipy.stats import spearmanr, pearsonr

import seaborn as sns
from matplotlib import pyplot as plt

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
def filtering(file_name):

  # Assume all the results file are located in this dir:
  loc = "/content/drive/MyDrive/IGOG-Alignment-Exp/Experiments/fan-effects-exp/results/rational/"

# Filter out the elements from the result:

  truep_t = []  # true present, typical
  truep_at = [] # true present, atypical
  falsep_t = []  # false present, typical
  falsep_at = []  # false present, atypical
  truea_t = []  # true absent, typical
  truea_at = [] # true absent, atypical
  falsea_t = []  # false absent, typical
  falsea_at = [] # false absent, atypical

  # List of typical members:
  typical_members = ["robin","sparrow","bluejay","bluebird","canary","blackbird","dove","lark","swallow","parakeet","oriole","mockingbird","redbird", "wren",
                    "finch","starling","cardinal","eagle","hummingbird","seagull","woodpecker","pigeon","thrush","falcon","crow","hawk","raven"]

  # List of atypical members:
  atypical_members = ["goldfinch","parrot","sandpiper","pheasant","catbird","crane","albatross","condor","toucan","owl","pelican","goose","vulture",
                      "stork","buzzard","swan","flamingo","duck","peacock","egret","chicken","turkey","ostrich","titmouse","emu","penguin","bat"]

  # Read the csv file:
  read_stimulus = pd.read_csv(loc+file_name,usecols=['stimulus'])
  read_item = pd.read_csv(loc+file_name,usecols=['item'])
  read_preamble = pd.read_csv(loc+file_name,usecols=['preamble'])
  read_score_val = pd.read_csv(loc+file_name,usecols=["score (sum, mean, [list)"])

  # Split preamble into a list of just only items
  read_preamble['split_preamble'] = read_preamble['preamble'].apply(lambda x: x.split(".")[0][15:].split(", "))

  for i in range(len(read_item)):
    item = read_item.at[i, 'item']
    # split_preamble = read_preamble['preamble'].apply(lambda x: x.split(".")[0][15:].split(", "))
    # split_preamble = split_preamble[15:].split(", ")
    split_preamble = read_preamble.at[i, 'split_preamble']
    # preamble = read_preamble[i, 'split_preamble']
    stimulus = read_stimulus.at[i, 'stimulus']
    score_val = read_score_val.at[i, "score (sum, mean, [list)"]

    if isinstance(score_val, str):
      # Extracting the mean value from the score string
      score_val = score_val.strip('[]()').split(',')[1].strip()
      score_val = float(score_val)

    if item in split_preamble and stimulus == "present.":  # true present
      if item in typical_members:
        truep_t.append(score_val)
      elif item in atypical_members:
        truep_at.append(score_val)
    elif item in split_preamble and stimulus == "absent.": # false absent
      if item in typical_members:
        falsea_t.append(score_val)
      elif item in atypical_members:
        falsea_at.append(score_val)
    elif item not in split_preamble and stimulus == "present.": # false present
      if item in typical_members:
        falsep_t.append(score_val)
      elif item in atypical_members:
        falsep_at.append(score_val)
    elif item not in split_preamble and stimulus == "absent.": # true absent
      if item in typical_members:
        truea_t.append(score_val)
      elif item in atypical_members:
        truea_at.append(score_val)
  # Visualization

  # Create a dictionary to store the lists
  map_df = {
      'true_p_typical': truep_t,
      'true_p_atypical': truep_at,
      'false_p_typical': falsep_t,
      'false_p_atypical': falsep_at,
      'true_a_typical': truea_t,
      'true_a_atypical': truea_at,
      'false_a_typical': falsea_t,
      'false_a_atypical': falsea_at
  }

  # Check the lengths of all lists
  list_lengths = [len(lst) for lst in map_df.values()]

  # Ensure all lengths are equal
  max_length = max(list_lengths)
  for lst in [truep_t, truep_at, falsep_t, falsep_at, truea_t, truea_at, falsea_t, falsea_at]:
      if len(lst) < max_length:
          # Fill missing values with NaN
          lst += [np.nan] * (max_length - len(lst))


  # Convert the dictionary to a DataFrame
  df = pd.DataFrame(map_df)
  print(df)


In [16]:
def main():
  print("Mistral")
  filtering('mistralai_Mistral-7B-v0.1.csv')
  print("Llama-2")
  filtering('meta-llama_Llama-2-7b-hf.csv')
  print("Llama-3")
  filtering('meta-llama_Meta-Llama-3-8B.csv')

if __name__ == "__main__":
  main()

Mistral
     true_p_typical  true_p_atypical  false_p_typical  false_p_atypical  \
0         -9.099177        -3.295715        -9.209574         -9.275753   
1         -8.862229        -3.891313        -8.461470         -8.345871   
2         -9.323246        -8.800889        -8.458887         -9.848536   
3        -10.051567        -7.738414        -8.897091         -9.486038   
4         -9.397378        -9.446735        -7.753295         -8.804101   
..              ...              ...              ...               ...   
398             NaN              NaN        -8.450175               NaN   
399             NaN              NaN        -9.474573               NaN   
400             NaN              NaN        -7.604613               NaN   
401             NaN              NaN       -10.382090               NaN   
402             NaN              NaN        -9.142367               NaN   

     true_a_typical  true_a_atypical  false_a_typical  false_a_atypical  
0        -10.1246

In [ ]:
def load_data(data_file, drop_cols=['preamble'], convert_prob=True):
    data = pd.read_csv(data_file, sep='|')
    data = data.drop(drop_cols, axis=1)

    try:
        data['stimulus'] = [[t for t in m] for m in data['stimulus'].apply(ast.literal_eval)]
    except ValueError as e:
        for m in data['stimulus_prob']:
            print(m)
        print(data['stimulus_prob'])
        return None

    if convert_prob:
        data['stimulus_prob'] = [[(2 ** t) for t in m] for m in data['stimulus_prob']]

    return pd.DataFrame(data)

In [ ]:
def split_confusion_matrix(df):
    test_pos = df.loc[(df['Dependent_Variable'] == 'present.')]
    test_neg = df.loc[(df['Dependent_Variable'] == 'absent.')]

    true_neg  = test_neg.loc[(test_neg['True_Category'] ==  'absent')].reset_index()
    false_pos = test_pos.loc[(test_pos['True_Category'] ==  'absent')].reset_index()
    false_neg = test_neg.loc[(test_neg['True_Category'] == 'present')].reset_index()
    true_pos  = test_pos.loc[(test_pos['True_Category'] == 'present')].reset_index()

    # Decided to return in the same order expected from sklearn confusion_matrix function
    return true_neg, false_pos, false_neg, true_pos

In [ ]:
def confusion_scatter(df, fan_type = 'both', aggregate='none'):
    if fan_type == 'both':
        fan = lambda a, b : (a*a + b*b)**(1/2)
    elif fan_type == 'first':
        fan = lambda a, b : a
    elif fan_type == 'second':
        fan = lambda a, b : b

    def fill_subplot(ax, df_part : pd.DataFrame, lab):
        x = []
        y = []

        for _, row in df_part.iterrows():
            fan_per = row['Fan_Person']
            fan_pla = row['Fan_Place']
            probs = row['stimulus_prob']

            if len(probs) != 50:
                raise ValueError

            if aggregate == 'none':
                x = x + [fan(fan_per, fan_pla)] * len(probs)
                y = y + probs
            elif aggregate == 'mean':
              x = x + [fan(fan_per, fan_pla)]
                y = y + [np.mean(probs)]
            elif aggregate == 'std':
                x = x + [fan(fan_per, fan_pla)]
                y = y + [np.std(probs)]

        x, y = np.asarray(x), np.asarray(y)

        m, b = np.polyfit(x, y, 1)
        # r, p = spearmanr(x, y)

        # z = spearmanr(x, y)
        z = pearsonr(x, y)
        r = z.statistic
        p = z.pvalue

        # ax.scatter(x,y, color=(0,0,0,max(0.1,1/len(x))))

        bins = dict()
        for x_i, y_i in zip(x,y):
          try:
                bins[x_i].append(y_i)
            except KeyError:
                bins[x_i] = [y_i]

        sorted_x = list(sorted(list(set(x))))
        new_y = [bins[k] for k in sorted_x]


        ax.violinplot(new_y, [50 * k for k in sorted_x])

        ax.plot(x, m*x + b, color='red')
        ax.text(0.01,0.01, f'r={r:.3f}, p={p:.3f} g={lab}', color='red', transform=ax.transAxes)

    tn, fp, fn, tp = split_confusion_matrix(df)

    fig, axes = plt.subplots(nrows = 2, ncols = 2, figsize=(15,15))

    fill_subplot(axes[0][0], tp, 'tp')
    # fill_subplot(axes[0][1], fn, 'fn')
    # fill_subplot(axes[1][0], fp, 'fp')
    # fill_subplot(axes[1][1], tn, 'tn')

    plt.show()


In [ ]:
def per_pop_corr (df, fan_type = 'both'):
    if fan_type == 'both':
        fan = lambda a, b : (a*a + b*b)**(1/2)
    elif fan_type == 'first':
        fan = lambda a, b : a
    elif fan_type == 'second':
        fan = lambda a, b : b

    _, _, _, df_part = split_confusion_matrix(df)

    # fig, ax = plt.subplots(nrows = 2, ncols = 2, figsize=(10,10))
    # fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(10,10))


    pop_size = len(df_part['stimulus_prob'][0])
    x = []
    y_list = [[] for _ in range(pop_size)]
    for _, row in df_part.iterrows():
        fan_per = row['Fan_Person']
        fan_pla = row['Fan_Place']
        probs = row['stimulus_prob']

        x = x + [fan(fan_per, fan_pla)]
        for i in range(pop_size):
            y_list[i].append(probs[i])

    # x, y = np.asarray(x), np.asarray(y)

    # m, b = np.polyfit(x, y, 1)

    rs, ps = [], []
    # z = spearmanr(x, y)
    # z = pearsonr(x, y)
    # r = z.statistic
    # p = z.pvalue

    for i in range(pop_size):
        # z = spearmanr(x, y_list[i])
        z = pearsonr(x, y_list[i])
        rs.append(z.statistic)
        ps.append(z.pvalue)

    return rs, ps

    ax.scatter(x,y, color=(0,0,0,max(0.1,1/len(x))))
    ax.plot(x, m*x + b, color='red')
    ax.text(0.01,0.01, f'r={r:.3f}, p={p:.3f} g=tp', color='red', transform=ax.transAxes)

    ax.set_xlabel('Fan Level')
    ax.set_ylabel('Probability')
    ax.set_xlim(0,None)
    ax.set_ylim(0,None)


    plt.show()

In [ ]:
models = [
    # 'LLaMa2-7B',
    # 'LLaMa3-8B',
    # 'LLaMa3-8B-Instruct',
    'Mistral-7B',
]

In [ ]:
for mod in models:
    df = load_data(f'../data/Random_TP_Only/results_{mod}.csv', convert_prob=True)
    exp_rs, exp_ps = per_pop_corr(df, fan_type='both')
    print(f'Model: {mod}\tfan type: both')
    print(np.asarray(exp_rs))
    print(np.mean(exp_rs))
    print(np.std(exp_rs))

    exp_rs, exp_ps = per_pop_corr(df, fan_type='first')
    print(f'Model: {mod}\tfan type: first')
    print(np.asarray(exp_rs))
    print(np.mean(exp_rs))
    print(np.std(exp_rs))

    exp_rs, exp_ps = per_pop_corr(df, fan_type='second')
    print(f'Model: {mod}\tfan type: second')
    print(np.asarray(exp_rs))
    print(np.mean(exp_rs))
    print(np.std(exp_rs))

    print()

In [ ]:
rand_mist = load_data('../data/Random/results_Mistral-7B.csv', convert_prob=True)

In [ ]:
rand_mist = load_data('../data/Random_TP_Only/results_Mistral-7B.csv', convert_prob=True)
confusion_scatter(rand_mist, fan_type='both', aggregate='std')
confusion_scatter(rand_mist, fan_type='first', aggregate='std')
confusion_scatter(rand_mist, fan_type='second', aggregate='std')